In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
COLOR_TREAT = "#2ecc71"
COLOR_NO_TREAT = "#e74c3c"

In [ ]:
lalonde = pd.read_csv('lalonde.csv')
#lalonde.set_index('id', drop=True, inplace=True)
lalonde

## 1. Naive analysis

TODO make a plot more comparable

In [ ]:
plt.figure(figsize=(12,5))

bins = np.histogram(lalonde.re78[lalonde['treat'] == 0])[1]
sns.distplot(lalonde.re78[lalonde['treat'] == 0], kde=False, color=COLOR_NO_TREAT, norm_hist=True, bins=bins)
sns.distplot(lalonde.re78[lalonde['treat'] == 1], kde=False, color=COLOR_TREAT, norm_hist=True, bins=bins)

plt.title('Distribution of the revenues')
plt.xlabel('Revenue in 1978')
plt.ylabel('Number of persons (Density)')
plt.legend(['No treatment', 'Treatment'])

Let's compare the average earnings of people that have/have not participated the job training:

In [ ]:
plt.figure(figsize=(20,5))
plt.subplot(121)
plt.title('Mean salary with/without training')
plt.ylabel('Mean salaray')
lalonde.groupby(['treat'])['re78'].mean().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])
plt.subplot(122)
plt.title('Median salary with/without training')
plt.ylabel('Median Salary')
lalonde.groupby(['treat'])['re78'].median().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])

## 2. A closer look at the data

In [ ]:
# distinguish categorical from non-categorical features
sns.set(font_scale=1.2)
lalonde_cat = lalonde[['black', 'hispan', 'married', 'nodegree', 'treat']]
lalonde_non_cat = lalonde[['age', 'educ', 're74', 're75', 're78', 'treat']]

### For each feature, compare its distribution in the treated group with its distribution in the control group

In [ ]:
f, axarr = plt.subplots(3, 3, figsize=(15, 18))
for index, feature in enumerate(['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78']):
    i = int(index/3)
    j = index%3
    bins = np.histogram(lalonde[feature][lalonde['treat'] == 0])[1]
    sns.distplot(lalonde[feature][lalonde['treat'] == 0], kde=False, color=COLOR_NO_TREAT, norm_hist=True, ax=axarr[i][j], bins=bins)
    sns.distplot(lalonde[feature][lalonde['treat'] == 1], kde=False, color=COLOR_TREAT, norm_hist=True, ax=axarr[i][j], bins=bins)
    axarr[i][j].set_title(feature)
    axarr[i][j].set_xlabel(feature)
    axarr[i][j].set_ylabel('Density')
    axarr[i][j].legend(['No treatment', 'Treatment'])

### Pairwise analysis of features' distribution
Comparing the distrbution of features for each distinct feature tuple can give additional information:
**TODO : décrire les obs**

In [ ]:
sns.pairplot(lalonde_non_cat, hue='treat', palette={0:"#e74c3c", 1: "#2ecc71"})

## 3. A propsensity score model
We should fit our model on the pre treatment features, though we will have to remove the re78 feature.

In [ ]:
import sklearn.linear_model

lal = lalonde.drop(['id','treat','re78'],1)

model = sklearn.linear_model.LogisticRegression()
model.fit(lal, lalonde.treat)
pred = model.predict_proba(lal)

In [ ]:
lalonde['pred'] = pred[:,1]
ax = sns.stripplot(x='id', y='pred', hue='treat', data=lalonde, palette={0:"#e74c3c", 1: "#2ecc71"})

## 4. Balancing the dataset via matching
Use the propensity scores to match each data point from the treated group with exactly one data point from the control group, while ensuring that each data point from the control group is matched with at most one data point from the treated group. (Hint: you may explore the networkx package in Python for predefined matching functions.)

Your matching should maximize the similarity between matched subjects, as captured by their propensity scores. In other words, the sum (over all matched pairs) of absolute propensity-score differences between the two matched subjects should be minimized.

After matching, you have as many treated as you have control subjects. Compare the outcomes (re78) between the two groups (treated and control).

Also, compare again the feature-value distributions between the two groups, as you've done in part 2 above, but now only for the matched subjects. What do you observe? Are you closer to being able to draw valid conclusions now than you were before?

In [ ]:
import networkx as nx
from networkx.algorithms import bipartite
#graph_size = lalonde['treat'].value_counts()
#G = nx.complete_bipartite_graph(graph_size[0], graph_size[1])
#G.add_nodes_from(lalonde['id'][lalonde.treat == 0], bipartite=0) # Add the node attribute "bipartite"
#G.add_nodes_from(lalonde['id'][lalonde.treat == 1], bipartite=1)

G=nx.Graph()
G.add_nodes_from(lalonde['id'][lalonde.treat == 0]) # Add the node attribute "bipartite"
G.add_nodes_from(lalonde['id'][lalonde.treat == 1])


In [ ]:
for ID_u, score_u in zip(lalonde.id[lalonde.treat == 0], lalonde.pred[lalonde.treat == 0]):
    for ID_v, score_v in zip(lalonde.id[lalonde.treat == 1], lalonde.pred[lalonde.treat == 1]):
        G.add_edge(ID_u, ID_v, weight=-abs(score_u-score_v))

In [ ]:
G.number_of_nodes()
nx.draw(G)

In [ ]:
from networkx.algorithms import max_weight_matching
matching = max_weight_matching(G, maxcardinality=True)

## 5. Balancing the groups further
The "balanced" mode uses the values of y to automatically adjust
    weights inversely proportional to class frequencies in the input data
    as ``n_samples / (n_classes * np.bincount(y))`

## 6. A less naive analysis